<a href="https://colab.research.google.com/github/Suhit97/Detector-Using-Tensorflow-Hub/blob/main/simple_detector_using_tensorflow_hub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from PIL import ImageOps
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO


## Download the model from Tensorflow Hub

you will want to look at the [image object detection subcategory](https://tfhub.dev/s?module-type=image-object-detection). 
 You can select a model to see more information about it and copy the URL so you can download it to your workspace.

In [ ]:
# inception resnet version 2
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"

# You can choose ssd mobilenet version 2 instead and compare the results
#module_handle = "https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1"

In [ ]:
model = hub.load(module_handle)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# take a look at the available signatures for this particular model
model.signatures.keys()

KeysView(_SignatureMap({'default': <ConcreteFunction pruned(images) at 0x7F7CB02BA8D0>}))

In [ ]:
#    For object detection models, its 'default' signature will accept 
# a batch of image tensors and output a dictionary describing the 
# objects detected, which is what you'll want here.

detector = model.signatures['default']

In [ ]:
# This function downloads an image specified by a given "url",
# pre-processes it, and then saves it to disk
def download_and_resize_image(url, new_width=256, new_height=256):
    
    
  
    # create a temporary file ending with ".jpg"
    _, filename = tempfile.mkstemp(suffix=".jpg")
    
    # opens the given URL
    response = urlopen(url)
    
    # reads the image fetched from the URL
    image_data = response.read()
    
    # puts the image data in memory buffer
    image_data = BytesIO(image_data)
    
    # opens the image
    pil_image = Image.open(image_data)
    
    # resizes the image. will crop if aspect ratio is different.
    pil_image = ImageOps.fit(pil_image, (new_width, new_height), Image.ANTIALIAS)
    
    # converts to the RGB colorspace
    pil_image_rgb = pil_image.convert("RGB")
    
    # saves the image to the temporary file created earlier
    pil_image_rgb.save(filename, format="JPEG", quality=90)
    
    print("Image downloaded to %s." % filename)
    
    return filename

In [ ]:
# You can choose a different URL that points to an image of your choice
image_url = "https://upload.wikimedia.org/wikipedia/commons/f/fb/20130807_dublin014.JPG"

# download the image and use the original height and width
downloaded_image_path = download_and_resize_image(image_url, 3872, 2592)

Image downloaded to /tmp/tmpdsd5e7ye.jpg.


In [ ]:
def load_img(path):

    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3) # convert to a tensor

    return img

def run_detector(detector, path):

    img = load_img(path)
    
    # add a batch dimension in front of the tensor
    converted_img  = tf.image.convert_image_dtype(img, tf.float32)[tf.newaxis, ...]
    
    # run inference using the model
    result = detector(converted_img)

    # save the results in a dictionary
    result = {key:value.numpy() for key,value in result.items()}

    print("Found %d objects." % len(result["detection_scores"]))

    print(result["detection_scores"])
    print(result["detection_class_entities"])
    print(result["detection_boxes"])

In [ ]:
# runs the object detection model and prints information about the objects found
run_detector(detector, downloaded_image_path)

Found 100 objects.
[0.6532174  0.61050546 0.6015252  0.59255606 0.5917772  0.58154905
 0.5505307  0.49575508 0.47424793 0.47322112 0.44066444 0.40511352
 0.39803842 0.3940648  0.37148526 0.3615601  0.3615082  0.34689075
 0.33362326 0.31252864 0.28877997 0.25758377 0.25748917 0.2519606
 0.24782118 0.23412398 0.20431991 0.20324546 0.1798833  0.17964837
 0.17374562 0.16431494 0.16031002 0.15895243 0.15620323 0.15468806
 0.14754413 0.13622656 0.12740128 0.12555613 0.12102727 0.11813042
 0.11387316 0.1122914  0.11129197 0.09718703 0.09137283 0.08976
 0.08880252 0.08633614 0.08337441 0.08095141 0.07988787 0.07741444
 0.07732119 0.0763102  0.07507841 0.07386033 0.07233331 0.07204077
 0.07110188 0.06935585 0.06825582 0.06427857 0.06248578 0.06226391
 0.06211119 0.05940086 0.05798771 0.05784462 0.05725606 0.0534669
 0.05304342 0.05242853 0.0489259  0.04813047 0.04578466 0.0442417
 0.0433756  0.04277815 0.0426263  0.04163285 0.04081782 0.03975986
 0.0394601  0.03944279 0.03863994 0.03769071 0.03